In [5]:
import pandas as pd
import numpy as np

import json

import datetime as dt

import os

In [6]:
df_station_locations = pd.read_csv(r"C:\Users\mkaya\OneDrive\Masaüstü\istanbul112_hidden\data\locations\station_locations.csv")


In [7]:
df_station_locations.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)

In [8]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [9]:
example_json_input = '''
    {
        "stations": [
            {
                "id": "BYR1",
                "region": "Beyoğlu",
                "importance": 5,
                "assignedPersonnel": [
                    {"id": "P001", "name": "Ahmet Yılmaz", "roles": ["medic"], "assignedFrom": "BYR1", "homeStationId": "BYR1", "negativeStations": ["BYR2"], "preferredStations": ["BYR1", "SIS1"], "rotationCount": 0},
                    {"id": "P002", "name": "Mehmet Kaya", "roles": ["medic"], "assignedFrom": "BYR1", "homeStationId": "BYR1", "negativeStations": [], "preferredStations": ["BYR1", "BYR3"], "rotationCount": 1},
                    {"id": "P003", "name": "Ali Demir", "roles": ["driver"], "assignedFrom": "BYR1", "homeStationId": "BYR1", "negativeStations": [], "preferredStations": [], "rotationCount": 0}
                ],
                "maxRotationCount": 2
            },
            {
                "id": "BYR2",
                "region": "Beyoğlu",
                "importance": 3,
                "assignedPersonnel": [
                    {"id": "P004", "name": "Ayşe Öztürk", "roles": ["medic"], "assignedFrom": "BYR2", "homeStationId": "BYR2", "negativeStations": [], "preferredStations": ["BYR2", "FTH1"], "rotationCount": 0},
                    {"id": "P005", "name": "Fatma Şahin", "roles": ["medic"], "assignedFrom": "BYR2", "homeStationId": "BYR2", "negativeStations": [], "preferredStations": ["BYR2"], "rotationCount": 2}
                ],
                "maxRotationCount": 3
            },
            {
                "id": "SIS1",
                "region": "Şişli",
                "importance": 4,
                "assignedPersonnel": [
                    {"id": "P006", "name": "Mustafa Çelik", "roles": ["medic", "driver"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": ["BYR2"], "preferredStations": ["SIS1", "SIS2"], "rotationCount": 1},
                    {"id": "P007", "name": "Zeynep Yıldız", "roles": ["medic"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": [], "preferredStations": ["SIS1", "BYR1"], "rotationCount": 0},
                    {"id": "P008", "name": "Hasan Arslan", "roles": ["driver"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": [], "preferredStations": [], "rotationCount": 3},
                    {"id": "P009", "name": "Osman Kara", "roles": ["medic"], "assignedFrom": "SIS1", "homeStationId": "SIS1", "negativeStations": [], "preferredStations": ["SIS1", "FTH1"], "rotationCount": 0}
                ],
                "maxRotationCount": 4
            },
            {
                "id": "FTH1",
                "region": "Fatih",
                "importance": 5,
                "assignedPersonnel": [
                    {"id": "P010", "name": "Emine Aydın", "roles": ["medic"], "assignedFrom": "FTH1", "homeStationId": "FTH1", "negativeStations": [], "preferredStations": ["FTH1"], "rotationCount": 1}
                ],
                "maxRotationCount": 2
            }
        ],
        neighboringRegions = {
            "Adalar": ["Kartal", "Pendik"],
            "Arnavutköy": ["Çatalca", "Esenler", "Başakşehir", "Eyüpsultan"],
            "Ataşehir": ["Üsküdar", "Kadıköy", "Maltepe", "Ümraniye"],
            "Avcılar": ["Küçükçekmece", "Bağcılar", "Başakşehir"],
            "Bağcılar": ["Küçükçekmece", "Bahçelievler", "Güngören", "Esenler", "Avcılar"],
            "Bahçelievler": ["Bağcılar", "Bakırköy", "Güngören"],
            "Bakırköy": ["Bahçelievler", "Güngören", "Küçükçekmece", "Esenyurt"],
            "Başakşehir": ["Arnavutköy", "Eyüpsultan", "Esenler", "Avcılar", "Küçükçekmece"],
            "Bayrampaşa": ["Gaziosmanpaşa", "Eyüpsultan", "Fatih", "Şişli"],
            "Beşiktaş": ["Şişli", "Beyoğlu", "Sarıyer", "Kağıthane"],
            "Beykoz": ["Ümraniye", "Çekmeköy", "Sancaktepe", "Üsküdar"],
            "Beylikdüzü": ["Büyükçekmece", "Esenyurt", "Avcılar"],
            "Beyoğlu": ["Şişli", "Kağıthane", "Fatih", "Beşiktaş", "Eyüpsultan"],
            "Büyükçekmece": ["Çatalca", "Silivri", "Esenyurt", "Beylikdüzü"],
            "Çatalca": ["Silivri", "Büyükçekmece", "Arnavutköy"],
            "Çekmeköy": ["Ümraniye", "Beykoz", "Üsküdar", "Sancaktepe"],
            "Esenler": ["Bağcılar", "Güngören", "Bakırköy", "Başakşehir", "Gaziosmanpaşa"],
            "Esenyurt": ["Küçükçekmece", "Avcılar", "Bakırköy", "Beylikdüzü", "Büyükçekmece"],
            "Eyüpsultan": ["Sarıyer", "Kağıthane", "Beyoğlu", "Gaziosmanpaşa", "Bayrampaşa", "Fatih", "Sultangazi", "Başakşehir", "Arnavutköy"],
            "Fatih": ["Beyoğlu", "Şişli", "Eminönü (merged into Fatih)", "Esenler?"],
            "Gaziosmanpaşa": ["Eyüpsultan", "Sultangazi", "Esenler", "Bayrampaşa"],
            "Güngören": ["Bağcılar", "Bahçelievler", "Bakırköy", "Esenler", "Zeytinburnu"],
            "Kadıköy": ["Ümraniye", "Ataşehir", "Maltepe", "Kartal", "Sancaktepe"],
            "Kağıthane": ["Şişli", "Beşiktaş", "Beyoğlu", "Eyüpsultan", "Sarıyer", "Sultangazi"],
            "Kartal": ["Maltepe", "Pendik", "Tuzla", "Adalar"],
            "Küçükçekmece": ["Avcılar", "Bağcılar", "Bakırköy", "Esenyurt", "Başakşehir"],
            "Maltepe": ["Kadıköy", "Kartal", "Pendik", "Sancaktepe", "Ümraniye"],
            "Pendik": ["Kartal", "Tuzla", "Sancaktepe", "Maltepe", "Ümraniye"],
            "Sancaktepe": ["Çekmeköy", "Ümraniye", "Kadıköy", "Maltepe", "Pendik", "Tuzla", "Kartal"],
            "Sarıyer": ["Eyüpsultan", "Kağıthane", "Beşiktaş", "Beykoz", "Şile"],
            "Silivri": ["Çatalca", "Büyükçekmece"],
            "Sultanbeyli": ["Pendik", "Kartal", "Sancaktepe", "Ümraniye"],
            "Sultangazi": ["Eyüpsultan", "Gaziosmanpaşa", "Kağıthane"],
            "Şile": ["Sarıyer"],
            "Şişli": ["Beyoğlu", "Beşiktaş", "Kağıthane", "Bayrampaşa", "Fatih"],
            "Tuzla": ["Pendik", "Kartal"],
            "Ümraniye": ["Küçükçekmece?", "Ataşehir", "Kadıköy", "Maltepe", "Sancaktepe", "Çekmeköy", "Üsküdar", "Beykoz"],
            "Üsküdar": ["Ümraniye", "Çekmeköy", "Beykoz", "Kadıköy", "Ataşehir"],
            "Zeytinburnu": ["Bakırköy", "Güngören", "Fatih", "Bayrampaşa"]
        },
        "maxRotationsPerPersonnel": 5
    }
    '''

In [10]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [11]:
def get_day_name_and_season(date):
    """
    Returns the current day name and season name.
    Day name is in uppercase (e.g., 'MONDAY').
    Season name is in uppercase (e.g., 'SUMMER').
    """
    date= pd.to_datetime(date, format='mixed')
    
    # Get day name
    day_name = date.strftime('%A') # e.g., 'WEDNESDAY'

    # Get season name
    def get_season(date):
        Y = date.year
        seasons = [
            ('Winter', dt.datetime(Y, 1, 1), dt.datetime(Y, 3, 21)),
            ('Spring', dt.datetime(Y, 3, 21), dt.datetime(Y, 6, 21)),
            ('Summer', dt.datetime(Y, 6, 21), dt.datetime(Y, 9, 23)),
            ('Autumn', dt.datetime(Y, 9, 23), dt.datetime(Y, 12, 21)),
            ('Winter', dt.datetime(Y, 12, 21), dt.datetime(Y+1, 1, 1))
        ]
        for season, start, end in seasons:
            if start <= date <= end:
                return season
        return 'UNKNOWN'

    season_name = get_season(date)

    return season_name, day_name


In [12]:
season, day= get_day_name_and_season('25-12-2025 14:45')

In [13]:
season

'Winter'

In [14]:
day

'Thursday'

In [15]:
json_input = '''
    {
        "stations": [
            {
                "id": "",
                "region": "",
                "importance": "",
                "assignedPersonnel": [
                    {"id": "", "name": "", "roles": [""], "assignedFrom": "", "homeStationId": "", "negativeStations": [], "preferredStations": [], "rotationCount": 0}
                ],
                "maxRotationCount": 5
            }
        ],
        "neighboringRegions" : {
            "Adalar": ["Kartal", "Pendik"],
            "Arnavutköy": ["Çatalca", "Esenler", "Başakşehir", "Eyüpsultan"],
            "Ataşehir": ["Üsküdar", "Kadıköy", "Maltepe", "Ümraniye"],
            "Avcılar": ["Küçükçekmece", "Bağcılar", "Başakşehir"],
            "Bağcılar": ["Küçükçekmece", "Bahçelievler", "Güngören", "Esenler", "Avcılar"],
            "Bahçelievler": ["Bağcılar", "Bakırköy", "Güngören"],
            "Bakırköy": ["Bahçelievler", "Güngören", "Küçükçekmece", "Esenyurt"],
            "Başakşehir": ["Arnavutköy", "Eyüpsultan", "Esenler", "Avcılar", "Küçükçekmece"],
            "Bayrampaşa": ["Gaziosmanpaşa", "Eyüpsultan", "Fatih", "Şişli"],
            "Beşiktaş": ["Şişli", "Beyoğlu", "Sarıyer", "Kağıthane"],
            "Beykoz": ["Ümraniye", "Çekmeköy", "Sancaktepe", "Üsküdar"],
            "Beylikdüzü": ["Büyükçekmece", "Esenyurt", "Avcılar"],
            "Beyoğlu": ["Şişli", "Kağıthane", "Fatih", "Beşiktaş", "Eyüpsultan"],
            "Büyükçekmece": ["Çatalca", "Silivri", "Esenyurt", "Beylikdüzü"],
            "Çatalca": ["Silivri", "Büyükçekmece", "Arnavutköy"],
            "Çekmeköy": ["Ümraniye", "Beykoz", "Üsküdar", "Sancaktepe"],
            "Esenler": ["Bağcılar", "Güngören", "Bakırköy", "Başakşehir", "Gaziosmanpaşa"],
            "Esenyurt": ["Küçükçekmece", "Avcılar", "Bakırköy", "Beylikdüzü", "Büyükçekmece"],
            "Eyüpsultan": ["Sarıyer", "Kağıthane", "Beyoğlu", "Gaziosmanpaşa", "Bayrampaşa", "Fatih", "Sultangazi", "Başakşehir", "Arnavutköy"],
            "Fatih": ["Beyoğlu", "Şişli", "Eminönü", "Esenler"],
            "Gaziosmanpaşa": ["Eyüpsultan", "Sultangazi", "Esenler", "Bayrampaşa"],
            "Güngören": ["Bağcılar", "Bahçelievler", "Bakırköy", "Esenler", "Zeytinburnu"],
            "Kadıköy": ["Ümraniye", "Ataşehir", "Maltepe", "Kartal", "Sancaktepe"],
            "Kağıthane": ["Şişli", "Beşiktaş", "Beyoğlu", "Eyüpsultan", "Sarıyer", "Sultangazi"],
            "Kartal": ["Maltepe", "Pendik", "Tuzla", "Adalar"],
            "Küçükçekmece": ["Avcılar", "Bağcılar", "Bakırköy", "Esenyurt", "Başakşehir"],
            "Maltepe": ["Kadıköy", "Kartal", "Pendik", "Sancaktepe", "Ümraniye"],
            "Pendik": ["Kartal", "Tuzla", "Sancaktepe", "Maltepe", "Ümraniye"],
            "Sancaktepe": ["Çekmeköy", "Ümraniye", "Kadıköy", "Maltepe", "Pendik", "Tuzla", "Kartal"],
            "Sarıyer": ["Eyüpsultan", "Kağıthane", "Beşiktaş", "Beykoz", "Şile"],
            "Silivri": ["Çatalca", "Büyükçekmece"],
            "Sultanbeyli": ["Pendik", "Kartal", "Sancaktepe", "Ümraniye"],
            "Sultangazi": ["Eyüpsultan", "Gaziosmanpaşa", "Kağıthane"],
            "Şile": ["Sarıyer"],
            "Şişli": ["Beyoğlu", "Beşiktaş", "Kağıthane", "Bayrampaşa", "Fatih"],
            "Tuzla": ["Pendik", "Kartal"],
            "Ümraniye": ["Üsküdar","Ataşehir", "Kadıköy", "Maltepe", "Sancaktepe", "Çekmeköy",  "Beykoz"],
            "Üsküdar": ["Ümraniye", "Çekmeköy", "Beykoz", "Kadıköy", "Ataşehir"],
            "Zeytinburnu": ["Bakırköy", "Güngören", "Fatih", "Bayrampaşa"]
        },
        "maxRotationsPerPersonnel": 5
    }
'''

In [16]:
data = json.loads(json_input)

In [17]:
data.keys()

dict_keys(['stations', 'neighboringRegions', 'maxRotationsPerPersonnel'])

In [18]:
pd.DataFrame(data['stations'])

,id,region,importance,assignedPersonnel,maxRotationCount
0,,,,"[{'id': '', 'name': '', 'roles': [''], 'assign...",5


In [19]:
data["stations"][0].keys()

dict_keys(['id', 'region', 'importance', 'assignedPersonnel', 'maxRotationCount'])

In [20]:

data["stations"][0]["assignedPersonnel"]

[{'id': '',
  'name': '',
  'roles': [''],
  'assignedFrom': '',
  'homeStationId': '',
  'negativeStations': [],
  'preferredStations': [],
  'rotationCount': 0}]

In [21]:
df_shifts= pd.read_excel(rf"C:\Users\mkaya\Downloads\Personel-Nöbet-Listesi (28).xls", header=None)

In [46]:
df_signs= pd.read_excel(r"C:\Users\mkaya\Downloads\Personel-Imza-Defteri (7).xls", header=None)

In [47]:
df_signs= df_signs[df_signs[2].notna()]

In [48]:
df_signs

,0,1,2,3,4,5,6,7,8,9,10,11
6,NaN,Sıra No,Adı Soyadı,NaN,NaN,Unvan,Başlama Tarihi,Giriş Açıklama,Bitiş Tarihi,NaN,Çıkış Açıklama,Ekip
7,NaN,1,ABDULKADİR ACET,NaN,NaN,Sürücü,23/06/2025 20:01:20,NaN,NaN,NaN,NaN,ARN3
8,NaN,2,ABDULKADİR GÜNDÜZ,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,AVRUPABAŞHEKİMLİK
9,NaN,3,ABDULKADİR ZORLU,NaN,NaN,Sürücü,23/06/2025 19:34:44,NaN,NaN,NaN,NaN,SRY4
10,NaN,4,ABDULLAH ATAN,NaN,NaN,Yardımcı Sağlık Personeli,NaN,NaN,NaN,NaN,NaN,OEUROLİNE
...,...,...,...,...,...,...,...,...,...,...,...,...
2007,NaN,2001,ZÜLBİYE SARI,NaN,NaN,Ekip Sorumlusu,23/06/2025 07:57:04,NaN,NaN,NaN,NaN,BÇK2
2008,NaN,2002,ZÜLEYHA KOÇ,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,AVRUPAİŞÇİSAĞLIĞI
2009,NaN,2003,ZÜLEYHA ŞENTÜRK,NaN,NaN,Diğer,NaN,NaN,NaN,NaN,NaN,AVUPAKALİTE
2010,NaN,2004,ZÜLFİYE DİLEK TAN,NaN,NaN,Yardımcı Sağlık Personeli,NaN,NaN,NaN,NaN,NaN,AVKKMACL


In [49]:
df_signs= df_signs.dropna(how='all',axis=1)

In [50]:
df_signs.columns= df_signs.iloc[0]


In [53]:
df_signs= df_signs[1:]

In [58]:
df_signs[(df_signs['Başlama Tarihi'].notna()) & ((pd.to_datetime(df_signs['Bitiş Tarihi'], format= 'mixed') - pd.to_datetime(df_signs['Başlama Tarihi'], format='mixed')) > pd.Timedelta(hours=8))]

6,Sıra No,Adı Soyadı,Unvan,Başlama Tarihi,Giriş Açıklama,Bitiş Tarihi,Çıkış Açıklama,Ekip
11,5,ABDULLAH AY,Sürücü,23/06/2025 08:05:44,NaN,24/06/2025 08:00:04,…,BDZ5
34,28,ADEM MUSULLU,Sürücü,23/06/2025 08:01:00,NaN,23/06/2025 20:47:44,.,BDZ4
43,37,AHMET AKAR,Sürücü,23/06/2025 08:05:09,Asos açılmadı 07:59 giriş yapıldı,23/06/2025 20:00:19,NaN,ARN1
54,48,AHMET ÇALIŞ,Sürücü,23/06/2025 19:51:18,NaN,24/06/2025 08:04:06,.,SLV5
63,57,AHMET EREN,Sürücü,23/06/2025 19:54:46,NaN,24/06/2025 08:28:31,....,ESY3
...,...,...,...,...,...,...,...,...
1972,1966,ZEHRA HAVAN,Yardımcı Sağlık Personeli,23/06/2025 09:05:50,NaN,24/06/2025 08:52:56,Ç,BAG6
1980,1974,ZELİHA YILMAZ,Diğer,23/06/2025 08:05:06,NaN,24/06/2025 08:10:13,NaN,KKM ÖZEL AMBULANS
1987,1981,ZEYNEP COŞKUN,Ekip Sorumlusu,23/06/2025 08:24:55,.,24/06/2025 08:30:42,.,BDZ8
1988,1982,ZEYNEP ÇINAR,Diğer,23/06/2025 07:42:10,\n,23/06/2025 18:20:47,Vh,AVC6


In [59]:
df_signs= df_signs[df_signs['Başlama Tarihi'].notna()]

In [43]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [62]:
# %load staff_shift_file_celaning.py
def staff_shift_file_cleaning(df):

    # WARNING: READ YOUR EXCEL FILE WITH header=None , F.e = pd.read_excel("df.xlsx", header=None)

    # Identify station rows: col1 has value, cols 2–4 are empty
    station_mask = df[1].notna() & df[[2, 3, 4]].isna().all(axis=1)
    df["RawStation"] = df[1].where(station_mask)

    # Forward fill station names
    df["İstasyon"] = df["RawStation"].ffill()
    df.columns= df.loc[df[df[1] == 'İsim'].index[0]]
    df= df[[col for col in df.columns if pd.notna(col)]]
    df.rename(columns={df.columns[-1]: 'İstasyon'}, inplace=True)
    df= df[(df['Kimlik No'].notna()) & (df['Kimlik No'] != 'Kimlik No')]
    
    return df

In [63]:
df_shifts= staff_shift_file_cleaning(df_shifts)

C:\Users\mkaya\AppData\Local\Temp\ipykernel_20164\2585991137.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={df.columns[-1]: 'İstasyon'}, inplace=True)


In [65]:
df_shifts['İsim Soyisim']= df_shifts['İsim'].astype(str).str.strip() + ' ' + df_shifts['Soyisim'].astype(str).str.strip()

In [68]:
df_signs['Adı Soyadı']= df_signs['Adı Soyadı'].astype(str).str.strip()

C:\Users\mkaya\AppData\Local\Temp\ipykernel_20164\1861579056.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_signs['Adı Soyadı']= df_signs['Adı Soyadı'].astype(str).str.strip()


In [71]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim
8,AHMET,TOSUN,13801994768,24 Saat,Diğer,YOK,657/4A Memur,05376703454,28-06-2025 08:00,29-06-2025 08:00,NaN,.ASKOM ARNAVUTKÖY,AHMET TOSUN
13,FATOŞ,CANYURT,51856389320,24 Saat,Diğer,YOK,657/4A Memur,05055024634,28-06-2025 08:00,29-06-2025 08:00,NaN,.ASKOM AVCILAR,FATOŞ CANYURT
19,UMUT HACİ,DEMİR,16762907884,24 Saat,Diğer,EVDE BAKIM,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05335665421,28-06-2025 08:00,29-06-2025 08:00,NaN,.ASKOM BAĞCILAR,UMUT HACİ DEMİR
24,ABDULLAH,BÜYÜKKAYA,25925261428,24 Saat,Diğer,YOK,657/4A Memur,05434995670,28-06-2025 08:00,29-06-2025 08:00,NaN,.ASKOM BAHÇELİEVLER,ABDULLAH BÜYÜKKAYA
29,MEVLÜDE,GÜL,14761501690,24 Saat,Diğer,Tıbbi Sekreter,657/4B Sözleşmeli Personel (663 sayılı KHK 45/A),05074368434,28-06-2025 08:00,29-06-2025 08:00,NaN,.ASKOM BAKIRKÖY*,MEVLÜDE GÜL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2058,TUĞBA,YAZICI,46312753876,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05432542208,28-06-2025 08:00,29-06-2025 08:00,NaN,Z.BURNU 6,TUĞBA YAZICI
2064,BATUHAN,ATİLLA,11360158140,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05321668024,28-06-2025 20:00,29-06-2025 08:00,NaN,Z.BURNU 7,BATUHAN ATİLLA
2065,ENGİN,KURT,20464784308,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05414237150,28-06-2025 08:00,28-06-2025 20:00,NaN,Z.BURNU 7,ENGİN KURT
2066,HASAN,YURTSEVEN,42991451522,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05382061731,28-06-2025 08:00,29-06-2025 08:00,NaN,Z.BURNU 7,HASAN YURTSEVEN


In [70]:
df_shifts[df_shifts['İsim Soyisim'].isin(df_signs['Adı Soyadı'])]

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,İsim Soyisim
93,BORA,TAŞÖREN,32689798350,12 Saat,Diğer,4132.05 VERİ GİRİŞ KONTROL İŞLETMENİ,Sürekli İşçi (696 sayılı KHK),05050141923,28-06-2025 20:00,29-06-2025 08:00,NaN,.ASKOM SAMATYA*,BORA TAŞÖREN
128,TUNAY,MINGIR,16675720228,24 Saat,Yardımcı Sağlık Personeli,İLK VE ACİL YARDIM,657/4A Memur,05079467893,28-06-2025 08:00,29-06-2025 08:00,NaN,.KRİZ BAKIRKÖY,TUNAY MINGIR
140,MURAT,ÖZAYDIN,58777401650,24 Saat,Yardımcı Sağlık Personeli,YOK,657/4A Memur,05557612457,28-06-2025 08:00,29-06-2025 08:00,NaN,.KRİZ ÇEKMECE,MURAT ÖZAYDIN
158,MUHAMMET,YILMAZ,31189066914,24 Saat,Yardımcı Sağlık Personeli,YOK,657/4A Memur,05377028862,28-06-2025 08:00,29-06-2025 08:00,NaN,.KRİZ KUZEY,MUHAMMET YILMAZ
164,ABDULKADİR,ACET,20066129736,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05453236502,28-06-2025 20:00,29-06-2025 08:00,NaN,ARNKÖY 1,ABDULKADİR ACET
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,MERT,YAVUZ,29764493958,24 Saat,Ekip Sorumlusu,İLK VE ACİL YARDIM,657/4A Memur,05369876562,28-06-2025 08:00,29-06-2025 08:00,NaN,SLTGAZİ 2,MERT YAVUZ
1896,AHMET,RENDECİ,18524772488,12 Saat,Sürücü,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05453009516,28-06-2025 20:00,29-06-2025 08:00,NaN,SLTGAZİ 3,AHMET RENDECİ
1897,OSMAN,ÖZGÜR,12038494652,12 Saat,Sürücü,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05337787989,28-06-2025 08:00,28-06-2025 20:00,NaN,SLTGAZİ 3,OSMAN ÖZGÜR
1941,YAKUP,GÜMÜŞ,49549137754,12 Saat,Sürücü,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05332319155,28-06-2025 08:00,28-06-2025 20:00,NaN,SLTGAZİ 8,YAKUP GÜMÜŞ


In [267]:
df_shifts.loc[(df_shifts['Görev']=='Ekip Sorumlusu') | (df_shifts['Görev']=='Yardımcı Sağlık Personeli'), 'Görev'] = 'medic'
df_shifts.loc[(df_shifts['Görev']=='Sürücü') , 'Görev'] = 'driver'

In [268]:
df_team_code_dict= pd.read_csv(r"C:\Users\mkaya\OneDrive\Masaüstü\istanbul112_hidden\data\keywords\team_code_match\eu_team_code.csv")
df_team_code_dict['112ONLINE']= df_team_code_dict['112ONLINE'].str.strip()
df_team_code_dict['ASOSYON']= df_team_code_dict['ASOS'].str.strip()

In [269]:
df_team_code_dict= dict(zip(df_team_code_dict['112ONLINE'], df_team_code_dict['ASOS']))

In [270]:
df_shifts['İstasyon']= df_shifts['İstasyon'].str.strip().map(df_team_code_dict)

In [271]:
df_shifts= df_shifts[df_shifts['İstasyon'].notna()]

In [272]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon
164,ABDULKADİR,ACET,20066129736,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05453236502,28-06-2025 20:00,29-06-2025 08:00,NaN,ARN1
165,NADA,ABDELHADİ,37247329434,24 Saat,medic,YOK,657/4A Memur,05378319286,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN1
171,ERHAN,KAVAK,16658937162,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372044593,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN2
172,RABİA,ÜNAL,15277780692,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05437608651,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN2
173,SERKAN,KAVAK,16655937226,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05307657872,28-06-2025 20:00,29-06-2025 08:00,NaN,ARN2
...,...,...,...,...,...,...,...,...,...,...,...,...
2058,TUĞBA,YAZICI,46312753876,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05432542208,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6
2064,BATUHAN,ATİLLA,11360158140,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05321668024,28-06-2025 20:00,29-06-2025 08:00,NaN,ZTB7
2065,ENGİN,KURT,20464784308,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05414237150,28-06-2025 08:00,28-06-2025 20:00,NaN,ZTB7
2066,HASAN,YURTSEVEN,42991451522,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05382061731,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB7


In [273]:
df_station_locations['Ekip Adı']= df_station_locations['Ekip Adı'].str.strip()

In [274]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [275]:
region_dict= dict(zip(df_station_locations['Ekip Adı'], df_station_locations['District']))

In [276]:
df_shifts['region']= df_shifts['İstasyon'].map(region_dict)

In [277]:
df_shifts= df_shifts[pd.to_datetime(df_shifts['Başlangıç Tarihi']).dt.hour < 11]

C:\Users\mkaya\AppData\Local\Temp\ipykernel_25704\885263242.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_shifts= df_shifts[pd.to_datetime(df_shifts['Başlangıç Tarihi']).dt.hour < 11]


In [278]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,region
165,NADA,ABDELHADİ,37247329434,24 Saat,medic,YOK,657/4A Memur,05378319286,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN1,Arnavutköy
171,ERHAN,KAVAK,16658937162,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372044593,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN2,Arnavutköy
172,RABİA,ÜNAL,15277780692,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05437608651,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN2,Arnavutköy
180,ERGİN,MERT,57373593844,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05300399874,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN3,Arnavutköy
181,SAKİNE,KOÇ,38524366838,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05521368840,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN3,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,HIDIR CAN,CÜCÜK,22441649822,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05061616587,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6,Zeytinburnu
2058,TUĞBA,YAZICI,46312753876,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05432542208,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6,Zeytinburnu
2065,ENGİN,KURT,20464784308,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05414237150,28-06-2025 08:00,28-06-2025 20:00,NaN,ZTB7,Zeytinburnu
2066,HASAN,YURTSEVEN,42991451522,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05382061731,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB7,Zeytinburnu


In [279]:
data

{'stations': [{'id': '',
   'region': '',
   'importance': '',
   'assignedPersonnel': [{'id': '',
     'name': '',
     'roles': [''],
     'assignedFrom': '',
     'homeStationId': '',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5}],
 'neighboringRegions': {'Adalar': ['Kartal', 'Pendik'],
  'Arnavutköy': ['Çatalca', 'Esenler', 'Başakşehir', 'Eyüpsultan'],
  'Ataşehir': ['Üsküdar', 'Kadıköy', 'Maltepe', 'Ümraniye'],
  'Avcılar': ['Küçükçekmece', 'Bağcılar', 'Başakşehir'],
  'Bağcılar': ['Küçükçekmece',
   'Bahçelievler',
   'Güngören',
   'Esenler',
   'Avcılar'],
  'Bahçelievler': ['Bağcılar', 'Bakırköy', 'Güngören'],
  'Bakırköy': ['Bahçelievler', 'Güngören', 'Küçükçekmece', 'Esenyurt'],
  'Başakşehir': ['Arnavutköy',
   'Eyüpsultan',
   'Esenler',
   'Avcılar',
   'Küçükçekmece'],
  'Bayrampaşa': ['Gaziosmanpaşa', 'Eyüpsultan', 'Fatih', 'Şişli'],
  'Beşiktaş': ['Şişli', 'Beyoğlu', 'Sarıyer', 'Kağıthane'],
  'Beykoz': 

In [280]:
season, day= get_day_name_and_season(df_shifts.iloc[0]["Başlangıç Tarihi"])

In [281]:
season, day

('Summer', 'Saturday')

In [282]:
df_shifts['Başlangıç Tarihi'].iloc[0]

'28-06-2025 08:00'

In [283]:
season, day

('Summer', 'Saturday')

In [284]:
scoring_file_name= f"{season}_{day}_total scores.xlsx"

In [285]:
df_scorings= pd.read_excel(rf"C:\Users\mkaya\OneDrive\Masaüstü\istanbul112_hidden\data\case_reports\europe\parquet_files\team_case_intensities\overall\{scoring_file_name}")

In [286]:
df_scorings

,Unnamed: 0,index,Ekip No,total_distance,field_operation_time,hospital_delivery_time,case_response_time,yas_score,cinsiyet_score,yeni_dogan_score,...,case_count_logarithmic_score_MinMax_score,case_count_logarithmic_score_MinMax_score_z_score,teams_population_density_logarithmic_score,teams_population_density_logarithmic_score_MinMax_score,teams_population_density_logarithmic_score_MinMax_score_z_score,total_score,strategic_score,total_score_z,strategic_score_z,station_expendable
0,0,99,ESY14,11.633071,1737.173420,745.546875,896.141935,39.087539,76.168224,25.000000,...,0.900457,1.316690,4.720795,0.823790,0.232733,23415.027627,2.390386,3.968752,-0.204869,Non-Expendable
1,1,102,ESY5,14.288632,1630.283304,759.941573,880.040395,38.854174,73.999148,25.000000,...,1.000000,3.336389,4.849307,0.912457,1.568817,23203.529808,1.803181,3.869731,-1.553061,Non-Expendable
2,2,94,ESY1,13.454183,1606.251976,723.840304,866.602193,37.439714,73.142857,25.047619,...,0.973539,2.799495,4.808144,0.884057,1.140867,21326.818052,1.835863,2.991077,-1.478026,Non-Expendable
3,3,144,KÇK11,14.149758,1871.750400,742.727799,1010.475962,42.207550,74.229584,25.000000,...,0.908161,1.473002,4.819280,0.891740,1.256642,20995.121823,2.491014,2.835781,0.026167,Non-Expendable
4,4,96,ESY11,12.447405,1810.818560,792.039724,929.897649,39.115415,75.461133,25.065876,...,0.931254,1.941560,4.762333,0.852449,0.664582,20540.683205,1.931773,2.623018,-1.257821,Non-Expendable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,194,ÇTL4,63.666667,2146.271186,1599.244444,1293.505747,45.306806,78.272251,25.000000,...,0.301509,-10.835791,3.841059,0.216815,-8.913511,6243.565194,12.391174,-4.070726,22.756425,Non-Expendable
201,201,193,ÇTL3,66.284133,2110.099265,2162.830409,1114.194030,48.388612,76.156584,25.000000,...,0.344372,-9.966112,3.856494,0.227465,-8.753034,5552.300374,9.902445,-4.394368,17.042432,Non-Expendable
202,202,174,SRY5,41.298013,3539.900332,1292.550562,1333.367133,37.470874,77.346278,25.000000,...,0.252526,-11.829641,4.630384,0.761411,-0.707235,5431.331147,6.865019,-4.451005,10.068657,Non-Expendable
203,203,113,EYP7,53.564767,3327.663043,1448.038168,1374.423913,39.943005,79.145078,25.000000,...,0.281180,-11.248254,4.013784,0.335987,-7.117759,4980.015421,8.016168,-4.662305,12.711637,Non-Expendable


In [287]:
data

{'stations': [{'id': '',
   'region': '',
   'importance': '',
   'assignedPersonnel': [{'id': '',
     'name': '',
     'roles': [''],
     'assignedFrom': '',
     'homeStationId': '',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5}],
 'neighboringRegions': {'Adalar': ['Kartal', 'Pendik'],
  'Arnavutköy': ['Çatalca', 'Esenler', 'Başakşehir', 'Eyüpsultan'],
  'Ataşehir': ['Üsküdar', 'Kadıköy', 'Maltepe', 'Ümraniye'],
  'Avcılar': ['Küçükçekmece', 'Bağcılar', 'Başakşehir'],
  'Bağcılar': ['Küçükçekmece',
   'Bahçelievler',
   'Güngören',
   'Esenler',
   'Avcılar'],
  'Bahçelievler': ['Bağcılar', 'Bakırköy', 'Güngören'],
  'Bakırköy': ['Bahçelievler', 'Güngören', 'Küçükçekmece', 'Esenyurt'],
  'Başakşehir': ['Arnavutköy',
   'Eyüpsultan',
   'Esenler',
   'Avcılar',
   'Küçükçekmece'],
  'Bayrampaşa': ['Gaziosmanpaşa', 'Eyüpsultan', 'Fatih', 'Şişli'],
  'Beşiktaş': ['Şişli', 'Beyoğlu', 'Sarıyer', 'Kağıthane'],
  'Beykoz': 

In [288]:
df_shifts["Görev"].unique()

array(['medic', 'driver', 'Refakatçi'], dtype=object)

In [289]:
def assignments(df_shifts):
    assignment_mask = np.select(
        [
            df_shifts["Görev"] == "Sürücü",
            (df_shifts["Görev"]=="Ekip Sorumlusu") | (df_shifts['Görev']=="Yardımcı Sağlık Personeli")
        ],
        [
            "driver",
            "medic"
        ],
        default=pd.NA
    )
    return assignment_mask

In [290]:
df_shifts['roles']= df_shifts.apply(assignments, axis=1)

C:\Users\mkaya\AppData\Local\Temp\ipykernel_25704\3402541198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shifts['roles']= df_shifts.apply(assignments, axis=1)


In [291]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,region,roles
165,NADA,ABDELHADİ,37247329434,24 Saat,medic,YOK,657/4A Memur,05378319286,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN1,Arnavutköy,<NA>
171,ERHAN,KAVAK,16658937162,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372044593,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN2,Arnavutköy,<NA>
172,RABİA,ÜNAL,15277780692,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05437608651,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN2,Arnavutköy,<NA>
180,ERGİN,MERT,57373593844,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05300399874,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN3,Arnavutköy,<NA>
181,SAKİNE,KOÇ,38524366838,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05521368840,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN3,Arnavutköy,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,HIDIR CAN,CÜCÜK,22441649822,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05061616587,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6,Zeytinburnu,<NA>
2058,TUĞBA,YAZICI,46312753876,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05432542208,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6,Zeytinburnu,<NA>
2065,ENGİN,KURT,20464784308,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05414237150,28-06-2025 08:00,28-06-2025 20:00,NaN,ZTB7,Zeytinburnu,<NA>
2066,HASAN,YURTSEVEN,42991451522,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05382061731,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB7,Zeytinburnu,<NA>


In [292]:
df_station_locations

,Ekip Adı,lat,lon,Bölge,city,district,station_count,rural,double_station,double_station_with,Neighbourhood,District
0,ARN1,41.177852,28.745802,ARN,ARNAVUTKÖY,ARNAVUTKÖY MERKEZ,1,KENTSEL,False,NaN,Arnavutköy Merkez Mahallesi,Arnavutköy
1,ARN2,41.156037,28.623824,ARN,ARNAVUTKÖY,HADIMKÖY,1,KENTSEL,False,NaN,Hadımköy Mahallesi,Arnavutköy
2,ARN3,41.204411,28.716732,ARN,ARNAVUTKÖY,MAREŞAL FEVZİ ÇAKMAK,1,KENTSEL,False,NaN,Mareşal Fevzi Çakmak Mahallesi,Arnavutköy
3,ARN4,41.199450,28.760639,ARN,ARNAVUTKÖY,HİCRET,1,KENTSEL,False,NaN,Hicret Mahallesi,Arnavutköy
4,ARN6,41.185858,28.708606,ARN,ARNAVUTKÖY,TAŞOLUK,1,KENTSEL,False,NaN,Taşoluk Mahallesi,Arnavutköy
...,...,...,...,...,...,...,...,...,...,...,...,...
207,ZTB7,41.006756,28.897603,ZTB,ZEYTİNBURNU,SEYİTNİZAM,1,KENTSEL,False,NaN,Seyitnizam Mahallesi,Zeytinburnu
208,ARN5,41.265160,28.688026,ARN,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,False,NaN,Tayakadın Mahallesi,Arnavutköy
209,ÇTL4,41.413472,28.186711,ÇTL,ÇATALCA,FATİH,1,KIRSAL,False,NaN,Fatih Mahallesi,Çatalca
210,EYP6,41.253788,28.714775,EYP,ARNAVUTKÖY,TAYAKADIN,1,KIRSAL,True,EYP7,Tayakadın Mahallesi,Arnavutköy


In [293]:

station_names= df_station_locations['Ekip Adı'].unique()

In [294]:
data['stations'][0]

{'id': '',
 'region': '',
 'importance': '',
 'assignedPersonnel': [{'id': '',
   'name': '',
   'roles': [''],
   'assignedFrom': '',
   'homeStationId': '',
   'negativeStations': [],
   'preferredStations': [],
   'rotationCount': 0}],
 'maxRotationCount': 5}

In [295]:
df_scorings

,Unnamed: 0,index,Ekip No,total_distance,field_operation_time,hospital_delivery_time,case_response_time,yas_score,cinsiyet_score,yeni_dogan_score,...,case_count_logarithmic_score_MinMax_score,case_count_logarithmic_score_MinMax_score_z_score,teams_population_density_logarithmic_score,teams_population_density_logarithmic_score_MinMax_score,teams_population_density_logarithmic_score_MinMax_score_z_score,total_score,strategic_score,total_score_z,strategic_score_z,station_expendable
0,0,99,ESY14,11.633071,1737.173420,745.546875,896.141935,39.087539,76.168224,25.000000,...,0.900457,1.316690,4.720795,0.823790,0.232733,23415.027627,2.390386,3.968752,-0.204869,Non-Expendable
1,1,102,ESY5,14.288632,1630.283304,759.941573,880.040395,38.854174,73.999148,25.000000,...,1.000000,3.336389,4.849307,0.912457,1.568817,23203.529808,1.803181,3.869731,-1.553061,Non-Expendable
2,2,94,ESY1,13.454183,1606.251976,723.840304,866.602193,37.439714,73.142857,25.047619,...,0.973539,2.799495,4.808144,0.884057,1.140867,21326.818052,1.835863,2.991077,-1.478026,Non-Expendable
3,3,144,KÇK11,14.149758,1871.750400,742.727799,1010.475962,42.207550,74.229584,25.000000,...,0.908161,1.473002,4.819280,0.891740,1.256642,20995.121823,2.491014,2.835781,0.026167,Non-Expendable
4,4,96,ESY11,12.447405,1810.818560,792.039724,929.897649,39.115415,75.461133,25.065876,...,0.931254,1.941560,4.762333,0.852449,0.664582,20540.683205,1.931773,2.623018,-1.257821,Non-Expendable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,200,194,ÇTL4,63.666667,2146.271186,1599.244444,1293.505747,45.306806,78.272251,25.000000,...,0.301509,-10.835791,3.841059,0.216815,-8.913511,6243.565194,12.391174,-4.070726,22.756425,Non-Expendable
201,201,193,ÇTL3,66.284133,2110.099265,2162.830409,1114.194030,48.388612,76.156584,25.000000,...,0.344372,-9.966112,3.856494,0.227465,-8.753034,5552.300374,9.902445,-4.394368,17.042432,Non-Expendable
202,202,174,SRY5,41.298013,3539.900332,1292.550562,1333.367133,37.470874,77.346278,25.000000,...,0.252526,-11.829641,4.630384,0.761411,-0.707235,5431.331147,6.865019,-4.451005,10.068657,Non-Expendable
203,203,113,EYP7,53.564767,3327.663043,1448.038168,1374.423913,39.943005,79.145078,25.000000,...,0.281180,-11.248254,4.013784,0.335987,-7.117759,4980.015421,8.016168,-4.662305,12.711637,Non-Expendable


In [296]:
data

{'stations': [{'id': '',
   'region': '',
   'importance': '',
   'assignedPersonnel': [{'id': '',
     'name': '',
     'roles': [''],
     'assignedFrom': '',
     'homeStationId': '',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5}],
 'neighboringRegions': {'Adalar': ['Kartal', 'Pendik'],
  'Arnavutköy': ['Çatalca', 'Esenler', 'Başakşehir', 'Eyüpsultan'],
  'Ataşehir': ['Üsküdar', 'Kadıköy', 'Maltepe', 'Ümraniye'],
  'Avcılar': ['Küçükçekmece', 'Bağcılar', 'Başakşehir'],
  'Bağcılar': ['Küçükçekmece',
   'Bahçelievler',
   'Güngören',
   'Esenler',
   'Avcılar'],
  'Bahçelievler': ['Bağcılar', 'Bakırköy', 'Güngören'],
  'Bakırköy': ['Bahçelievler', 'Güngören', 'Küçükçekmece', 'Esenyurt'],
  'Başakşehir': ['Arnavutköy',
   'Eyüpsultan',
   'Esenler',
   'Avcılar',
   'Küçükçekmece'],
  'Bayrampaşa': ['Gaziosmanpaşa', 'Eyüpsultan', 'Fatih', 'Şişli'],
  'Beşiktaş': ['Şişli', 'Beyoğlu', 'Sarıyer', 'Kağıthane'],
  'Beykoz': 

In [297]:
station_names[1]

'ARN2'

In [298]:
df_shifts

7,İsim,Soyisim,Kimlik No,Mesai,Görev,Branş,Kadro,Telefon,Başlangıç Tarihi,Bitiş Tarihi,Açıklama,İstasyon,region,roles
165,NADA,ABDELHADİ,37247329434,24 Saat,medic,YOK,657/4A Memur,05378319286,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN1,Arnavutköy,<NA>
171,ERHAN,KAVAK,16658937162,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05372044593,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN2,Arnavutköy,<NA>
172,RABİA,ÜNAL,15277780692,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05437608651,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN2,Arnavutköy,<NA>
180,ERGİN,MERT,57373593844,12 Saat,driver,8322.04 AMBULANS SÜRÜCÜSÜ,Sürekli İşçi (696 sayılı KHK),05300399874,28-06-2025 08:00,28-06-2025 20:00,NaN,ARN3,Arnavutköy,<NA>
181,SAKİNE,KOÇ,38524366838,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05521368840,28-06-2025 08:00,29-06-2025 08:00,NaN,ARN3,Arnavutköy,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2056,HIDIR CAN,CÜCÜK,22441649822,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05061616587,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6,Zeytinburnu,<NA>
2058,TUĞBA,YAZICI,46312753876,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05432542208,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB6,Zeytinburnu,<NA>
2065,ENGİN,KURT,20464784308,12 Saat,driver,AMBULANS SÜRÜCÜSÜ,Sürekli İşçi,05414237150,28-06-2025 08:00,28-06-2025 20:00,NaN,ZTB7,Zeytinburnu,<NA>
2066,HASAN,YURTSEVEN,42991451522,24 Saat,medic,İLK VE ACİL YARDIM,657/4A Memur,05382061731,28-06-2025 08:00,29-06-2025 08:00,NaN,ZTB7,Zeytinburnu,<NA>


C:\Users\mkaya\AppData\Local\Temp\ipykernel_25704\1175906179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shifts['İsim Soyisim']= df_shifts['İsim'].astype(str).str.strip() + ' ' + df_shifts['Soyisim'].astype(str).str.strip()


In [300]:
df_shifts['Kimlik No']= df_shifts['Kimlik No'].astype('int64')

C:\Users\mkaya\AppData\Local\Temp\ipykernel_25704\3537363259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shifts['Kimlik No']= df_shifts['Kimlik No'].astype('int64')


In [301]:
json_input = '''
    {
        "stations": [
            {
                "id": "",
                "region": "",
                "importance": "",
                "assignedPersonnel": [
                    {"id": "", "name": "", "roles": [""], "assignedFrom": "", "homeStationId": "", "negativeStations": [], "preferredStations": [], "rotationCount": 0}
                ],
                "maxRotationCount": 5
            }
        ],
        "neighboringRegions" : {
            "Adalar": ["Kartal", "Pendik"],
            "Arnavutköy": ["Çatalca", "Esenler", "Başakşehir", "Eyüpsultan"],
            "Ataşehir": ["Üsküdar", "Kadıköy", "Maltepe", "Ümraniye"],
            "Avcılar": ["Küçükçekmece", "Bağcılar", "Başakşehir"],
            "Bağcılar": ["Küçükçekmece", "Bahçelievler", "Güngören", "Esenler", "Avcılar"],
            "Bahçelievler": ["Bağcılar", "Bakırköy", "Güngören"],
            "Bakırköy": ["Bahçelievler", "Güngören", "Küçükçekmece", "Esenyurt"],
            "Başakşehir": ["Arnavutköy", "Eyüpsultan", "Esenler", "Avcılar", "Küçükçekmece"],
            "Bayrampaşa": ["Gaziosmanpaşa", "Eyüpsultan", "Fatih", "Şişli"],
            "Beşiktaş": ["Şişli", "Beyoğlu", "Sarıyer", "Kağıthane"],
            "Beykoz": ["Ümraniye", "Çekmeköy", "Sancaktepe", "Üsküdar"],
            "Beylikdüzü": ["Büyükçekmece", "Esenyurt", "Avcılar"],
            "Beyoğlu": ["Şişli", "Kağıthane", "Fatih", "Beşiktaş", "Eyüpsultan"],
            "Büyükçekmece": ["Çatalca", "Silivri", "Esenyurt", "Beylikdüzü"],
            "Çatalca": ["Silivri", "Büyükçekmece", "Arnavutköy"],
            "Çekmeköy": ["Ümraniye", "Beykoz", "Üsküdar", "Sancaktepe"],
            "Esenler": ["Bağcılar", "Güngören", "Bakırköy", "Başakşehir", "Gaziosmanpaşa"],
            "Esenyurt": ["Küçükçekmece", "Avcılar", "Bakırköy", "Beylikdüzü", "Büyükçekmece"],
            "Eyüpsultan": ["Sarıyer", "Kağıthane", "Beyoğlu", "Gaziosmanpaşa", "Bayrampaşa", "Fatih", "Sultangazi", "Başakşehir", "Arnavutköy"],
            "Fatih": ["Beyoğlu", "Şişli", "Eminönü", "Esenler"],
            "Gaziosmanpaşa": ["Eyüpsultan", "Sultangazi", "Esenler", "Bayrampaşa"],
            "Güngören": ["Bağcılar", "Bahçelievler", "Bakırköy", "Esenler", "Zeytinburnu"],
            "Kadıköy": ["Ümraniye", "Ataşehir", "Maltepe", "Kartal", "Sancaktepe"],
            "Kağıthane": ["Şişli", "Beşiktaş", "Beyoğlu", "Eyüpsultan", "Sarıyer", "Sultangazi"],
            "Kartal": ["Maltepe", "Pendik", "Tuzla", "Adalar"],
            "Küçükçekmece": ["Avcılar", "Bağcılar", "Bakırköy", "Esenyurt", "Başakşehir"],
            "Maltepe": ["Kadıköy", "Kartal", "Pendik", "Sancaktepe", "Ümraniye"],
            "Pendik": ["Kartal", "Tuzla", "Sancaktepe", "Maltepe", "Ümraniye"],
            "Sancaktepe": ["Çekmeköy", "Ümraniye", "Kadıköy", "Maltepe", "Pendik", "Tuzla", "Kartal"],
            "Sarıyer": ["Eyüpsultan", "Kağıthane", "Beşiktaş", "Beykoz", "Şile"],
            "Silivri": ["Çatalca", "Büyükçekmece"],
            "Sultanbeyli": ["Pendik", "Kartal", "Sancaktepe", "Ümraniye"],
            "Sultangazi": ["Eyüpsultan", "Gaziosmanpaşa", "Kağıthane"],
            "Şile": ["Sarıyer"],
            "Şişli": ["Beyoğlu", "Beşiktaş", "Kağıthane", "Bayrampaşa", "Fatih"],
            "Tuzla": ["Pendik", "Kartal"],
            "Ümraniye": ["Üsküdar","Ataşehir", "Kadıköy", "Maltepe", "Sancaktepe", "Çekmeköy",  "Beykoz"],
            "Üsküdar": ["Ümraniye", "Çekmeköy", "Beykoz", "Kadıköy", "Ataşehir"],
            "Zeytinburnu": ["Bakırköy", "Güngören", "Fatih", "Bayrampaşa"]
        },
        "maxRotationsPerPersonnel": 5
    }
'''

In [302]:
data = json.loads(json_input)

In [303]:
for station_name in station_names:
    df_assignments = df_shifts[df_shifts['İstasyon'] == station_name].copy()

    data['stations'][-1]['id'] = station_name

    # Get region as string
    region_series = df_station_locations[df_station_locations['Ekip Adı'] == station_name]['District']
    data['stations'][-1]['region'] = region_series.iloc[0] if not region_series.empty else ""

    # Get importance as float (fix here!)
    importance_series = df_scorings[df_scorings['Ekip No'] == station_name]['total_score_z']
    data['stations'][-1]['importance'] = float(importance_series.iloc[0]) if not importance_series.empty else 0.0
    
    added_worker_ids= []
    
    for i in range(len(df_assignments)):
        worker_id= df_assignments.iloc[i]['Kimlik No']
        if worker_id  not in added_worker_ids:
            added_worker_ids.append(df_assignments.iloc[i]['Kimlik No'])
            data['stations'][-1]['assignedPersonnel'][-1]['id'] = str(worker_id)
            data['stations'][-1]['assignedPersonnel'][-1]['name'] = str(df_assignments.iloc[i]['İsim Soyisim'])
            data['stations'][-1]['assignedPersonnel'][-1]['roles'][0] = str(df_assignments.iloc[i]['Görev'])
            data['stations'][-1]['assignedPersonnel'][-1]['homeStationId'] = str(df_assignments.iloc[i]['İstasyon'])
            data['stations'][-1]['assignedPersonnel'][-1]['assignedFrom']= str(df_assignments.iloc[i]['İstasyon'])
        else:
            print(f"Worker id {worker_id} is already assigned in another station")
            
        if i != len(df_assignments) - 1:            
            data['stations'][-1]['assignedPersonnel'].append({
                'id': '',
                'name': '',
                'roles': [''],
                'assignedFrom': '',
                'homeStationId': '',
                'negativeStations': [],
                'preferredStations': [],
                'rotationCount': 0})
        else:
            pass
    if station_name != station_names[-1]:      
        data['stations'].append({
        'id': '',
        'region': '',
        'importance': '',
        'assignedPersonnel': [{'id': '',
        'name': '',
        'roles': [''],
        'assignedFrom': '',
        'homeStationId': '',
        'negativeStations': [],
        'preferredStations': [],
        'rotationCount': 0}],
        'maxRotationCount': 2})
    else:
        pass

In [304]:
with open('C:/Users/mkaya/Onedrive/Masaüstü/data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [3]:
with open('C:/Users/mkaya/Onedrive/Masaüstü/data.json', 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

In [4]:
loaded_data

{'stations': [{'id': 'ARN1',
   'region': 'Arnavutköy',
   'importance': 0.839274232630034,
   'assignedPersonnel': [{'id': '37247329434',
     'name': 'NADA ABDELHADİ',
     'roles': ['medic'],
     'assignedFrom': 'ARN1',
     'homeStationId': 'ARN1',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 5},
  {'id': 'ARN2',
   'region': 'Arnavutköy',
   'importance': -1.726536132201086,
   'assignedPersonnel': [{'id': '16658937162',
     'name': 'ERHAN KAVAK',
     'roles': ['driver'],
     'assignedFrom': 'ARN2',
     'homeStationId': 'ARN2',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0},
    {'id': '15277780692',
     'name': 'RABİA ÜNAL',
     'roles': ['medic'],
     'assignedFrom': 'ARN2',
     'homeStationId': 'ARN2',
     'negativeStations': [],
     'preferredStations': [],
     'rotationCount': 0}],
   'maxRotationCount': 2},
  {'id': 'ARN3',
   'region': 'Arnavutköy',
   'importanc